In [11]:
import os
import sqlite3

import pandas as pd

In [5]:
# Recall the table structure of our new combined research database

patients = ['patient_id', 'dob', 'gender', 'race', 'postal_code', 'occupation'] 
admissions = ['patient_id', 'hadm_id', 'intime', 'outtime']
procedures = ['hadm_id', 'icd9_code'] 
diagnoses = ['hadm_id', 'icd9_code']
chartevents = ['hadm_id', 'item_id', 'value', 'datetime']
bpstream = ['patient_id', 'datetime', 'bp_min', 'bp_max']
medications = ['patient_id', 'datetime', 'drug']

# And the following dictionaries for codes
d_procedures = {3000:'Carotid endarterectomy'}
d_diagnoses = {10000:'diabetes', 10001:'hiv', 10002:'cancer'}
d_chartevents = {2000:'weight', 2001:'abp'}

In [6]:
# We want to end up with one table of features. Each row contains all the features 
# for one patient

# Final desired features (and outcome) for each patient.

features = [
    'age', 'gender', 'race', 'postal_code', 'occupation',
    # diagnoses
    'diabetes', 'hiv', 'cancer',
    # chartevents
    'weight_hosp', 'bp_hosp_min', 'bp_hosp_max',
    # bp numerics from wearables
    'bp_disch_min', 'bp_disch_max',
    # medications
    'dexamethasone', 'erlotinib'
    # outcome
    'readmission'
]

In [47]:
# Data extraction. Cohort: Patients who received carotid endartorectomy

query = """
-- Get the hadms with the endartorectory procedure
with t0 as(
    select pr.hadm_id, a.intime, a.outtime, a.patient_id
    from procedures pr
    left join admissions a on pr.hadm_id = a.hadm_id
    where pr.icd9_code = 3000
),
-- Get the demographic info of the patients
t1 as( 
    select t0.patient_id, t0.hadm_id, t0.intime, t0.outtime,
        p.gender, p.race, p.postal_code, p.occupation,
        Cast((JulianDay(t0.outtime) - JulianDay(p.dob)) / 365 as float) age
    from t0
    left join patients p on t0.patient_id = p.patient_id
),
-- Get the weights
weights as(
    select hadm_id, max(value) weight
    from chartevents
    where item_id == 2000
    group by hadm_id
),
-- Get the max and min bps during the admission
bphospital as( 
    select hadm_id, max(value) bp_hosp_max, min(value) bp_hosp_min
    from chartevents
    where item_id == 2001
    group by hadm_id
),
-- For each hadm_id, get whether diagnoses were made
comorbidities as (  
    select hadm_id, 
        max(case icd9_code when 10000 then 1 else 0 end) diabetes,
        max(case icd9_code when 10001 then 1 else 0 end) hiv,
        max(case icd9_code when 10002 then 1 else 0 end) cancer
    from diagnoses
    group by hadm_id
),
-- Get the desired medications
drugs as(
    select patient_id, datetime,
        max(case drug when 'Dexamethasone' then 1 else 0 end) dexamethasone,
        max(case drug when 'Erlotinib' then 1 else 0 end) erlotinib
    from medications
    group by patient_id
),
-- Join the wearable bp hourly numeric measurements on hadm_id
t2 as(
    select t1.*, b.datetime bptime, b.bp_min bp_disch_min, b.bp_max bp_disch_max
    from t1
    left join bpstream b
    on b.patient_id = t1.patient_id
),
-- Keep only bp measurements occuring after discharge time within 30 days
-- and get max/min out of those
t3 as(
    select patient_id, hadm_id, gender, race, postal_code, occupation, intime, outtime,
        max(bp_disch_max) bp_disch_max,
        min(bp_disch_min) bp_disch_min
    from t2
    where Cast((JulianDay(bptime) - JulianDay(outtime)) as float) between 0 and 30
    group by hadm_id
),
-- Get outcome: 30 day readmission
-- Start by getting all admission times, joining on patient_id
t4 as(
    select patient_id, intime
    from admissions
),
t5 as(
    select t3.*, (t4.intime is not null) readmission
    from t3
    left join t4 
    on t3.patient_id = t4.patient_id
        and t4.intime > t3.outtime
)
-- Join on the other features we generated earlier.
-- For comorbidities and drugs, not all hadm/patient ids have entries
select gender, race, postal_code, occupation,
    weight,
    bp_hosp_min, bp_hosp_max,
    case diabetes when diabetes is not null then diabetes else 0 end diabetes,
    case hiv when hiv is not null then hiv else 0 end hiv,
    case cancer when cancer is not null then cancer else 0 end cancer,
    case dexamethasone when dexamethasone is not null then dexamethasone else 0 end dexamethasone,
    case erlotinib when erlotinib is not null then erlotinib else 0 end erlotinib,
    bp_disch_min, bp_disch_max,
    readmission
from t5
left join bphospital bph
on t5.hadm_id = bph.hadm_id
left join weights w
on t5.hadm_id = w.hadm_id
left join comorbidities c
on t5.hadm_id = c.hadm_id
left join drugs d
on t5.patient_id = d.patient_id;
"""

In [48]:
conn = sqlite3.connect('data/allhealth.db')
df_features = pd.read_sql_query(query, conn)
conn.close()

In [49]:
df_features.head()

,gender,race,postal_code,occupation,weight,bp_hosp_min,bp_hosp_max,diabetes,hiv,cancer,dexamethasone,erlotinib,bp_disch_min,bp_disch_max,readmission
0,female,black,42956,builder,115,64,88,1,0,0,1,0,40,109,0
1,male,white,40369,engineer,112,66,75,0,0,1,0,1,40,108,0
2,female,asian,44890,doctor,46,55,80,0,1,1,0,0,41,106,0
3,female,native_american,40614,builder,104,41,103,0,1,1,0,0,40,109,1
4,male,asian,43761,firefighter,67,49,103,0,0,0,1,0,40,109,0


In [50]:
# Write the content to a flat csv file
df_features.to_csv('data/study-table.csv', index=False)

In [4]:
from sklearn import svm, neighbors
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

In [ ]:
# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features[:, :-1], features[:, -1],
                                                    train_size=0.75, test_size=0.25,
                                                    random_state=0)
print('Number of training records: %d' % len(x_train))
print('Number of testing records: %d' % len(x_test))
